<a href="https://colab.research.google.com/github/GustavoBrito99/Text-Mining/blob/main/TMProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [4]:
!pip install fsspec[http]>=2021.4.0
!pip install torchvision==0.2.2
!pip install unbabel-comet
!pip install -U -q PyDrive
!pip install rouge-score
!pip install fairseq
!pip install torch==1.4.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install unbabel-comet==0.1.0 torch==1.4.0

     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.19.9 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement tqdm>=4.36.1, but you'll have tqdm 4.19.9 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
     |████████████████████████████████| 61kB 4.1MB/s 
     |████████████████████████████████| 808kB 36.2MB/s 
     |████████████████████████████████| 748.8MB 22kB/s 
     |████████████████████████████████| 25.9MB 124kB/s 
     |████████████████████████████████| 307kB 54.9MB/s 
     |████████████████████████████████| 22.3MB 127kB/s 
     |████████████████████████████████| 2.8MB 38.0MB/s 
     |█████████████████

In [18]:
import pandas as pd
import numpy as np
import re 
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer
from nltk.util import ngrams
from nltk import skipgrams
from bs4 import BeautifulSoup
from scipy import stats
import string
from tqdm import tqdm_notebook as tqdm
from rouge_score import rouge_scorer
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from nltk.translate import chrf_score

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
import string

import nltk
nltk.download('stopwords')

import nltk
nltk.download('wordnet')

import nltk
nltk.download('punkt')
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

snowball_stemmer = SnowballStemmer('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
downloaded = drive.CreateFile({'id':"1QKBIH9btzyiX7tarXSZEM7Oym1DvlrAa"})   # replace the id with id of file you want to access
downloaded.GetContentFile('en_fi_scores.csv')
enfi = pd.read_csv('en_fi_scores.csv')

In [7]:
downloaded = drive.CreateFile({'id':"1zT_SYMi9akbV_gl2kP3hZu2SU64ll-s2"})   # replace the id with id of file you want to access
downloaded.GetContentFile('ru_en_scores.csv')
ruen = pd.read_csv('ru_en_scores.csv')

In [8]:
downloaded = drive.CreateFile({'id':"1xMv65gPcJogFpzcSokvvAi1673-4bWcI"})   # replace the id with id of file you want to access
downloaded.GetContentFile('en_zh_scores.csv')
enzh = pd.read_csv('en_zh_scores.csv')

In [9]:
downloaded = drive.CreateFile({'id':"1wYwZ633QmS0z3cPskd06-ClmcYjx6Bhj"})   # replace the id with id of file you want to access
downloaded.GetContentFile('zh_en_scores.csv')
zhen = pd.read_csv('zh_en_scores.csv')

In [10]:
downloaded = drive.CreateFile({'id':"1D2AIcC82QJoOjF_fQQUC7bEr17YMSSfj"})   # replace the id with id of file you want to access
downloaded.GetContentFile('de_en_scores.csv')
deen = pd.read_csv('de_en_scores.csv')

In [11]:
downloaded = drive.CreateFile({'id':"18U6ktF3bcVs-pGXdVLLM7j8hiFgxEJxc"})   # replace the id with id of file you want to access
downloaded.GetContentFile('cs_en_scores.csv')
csen = pd.read_csv('cs_en_scores.csv')

#Functions


In [ ]:
def n_gram(refs, n):
        # tokenize the references and the candidate
        refs = [ref.split() for ref in refs]
        output = list(ngrams(refs[0], n))
        return output

def skip_gram(refs, n, k):
        # tokenize the references and the candidate
        
        refs = [ref.split() for ref in refs]
        output = list(skipgrams(refs[0], n, k))
        
        return output

def BLEU_star_compact(refs, candidate, n): #can also be used for rouge precision
    
    refs = [n_gram(refs, n)]
    candidate = n_gram([candidate], n)

    return sum([min(count, max([ref[word] for ref in [Counter(ref) for ref in refs]])) for word, count in Counter(candidate).items()])/len(candidate)

def BLEU_4_avg(refs, candidate):
    avg = 0
    
    for i in range(4):
        if len(n_gram([candidate], i+1)) == 0: #If the translation (there were no problems with the reference) has less words than n
            return avg/(i+1)
        else:
            avg+=BLEU_star_compact(refs, candidate, i+1)
    return avg/4

def rouge_fscore(ref, candidates, n):
    
    def Rouge_recall(refs, candidate, n):
    
        refs = [n_gram(refs, n)]
        candidate = n_gram([candidate], n)

        return sum([min(count, max([ref[word] for ref in [Counter(ref) for ref in refs]])) for word, count in Counter(candidate).items()])/len(refs[0])

    precision = BLEU_star_compact(ref, candidates, n)
    recall = Rouge_recall(ref, candidates, n)
    if precision+recall == 0: #Can´t divide by zero
        return 0
    else:
        f1= 2*((precision*recall)/(precision+recall))
        return f1

def rouge_fscore_4_avg(refs, candidate):
    avg = 0
    
    for i in range(4):
        if len(n_gram([candidate], i+1)) == 0 or len(n_gram(refs, i+1)) == 0: #If the translation and reference have less words than n
            return avg/(i+1)
        else:
            avg+=rouge_fscore(refs, candidate, i+1)
    return avg/4

#RougeS (Not exactly RougeS)

def Rouge_skipgrams_precision(refs, candidate, n, k):
    #can also be used for rouge precision
    
    refs = [skip_gram(refs, n, k)]
    candidate = skip_gram([candidate], n, k)

    return sum([min(count, max([ref[word] for ref in [Counter(ref) for ref in refs]])) for word, count in Counter(candidate).items()])/len(candidate)

def Rouge_skipgrams_recall(refs, candidate, n, k):
    #can also be used for rouge precision
    
    refs = [skip_gram(refs, n, k)]
    candidate = skip_gram([candidate], n, k)

    return sum([min(count, max([ref[word] for ref in [Counter(ref) for ref in refs]])) for word, count in Counter(candidate).items()])/len(refs)

def rouge_final_S(precision, recall):
    f1= 2*(precision*recall)/(precision+recall)
    return f1


#Pre-Processing Functions


def clean(text_list, lemmatize, stemmer):
    """
    Function that a receives a list of strings and preprocesses it.
    
    :param text_list: List of strings.
    :param lemmatize: Tag to apply lemmatization if True.
    :param stemmer: Tag to apply the stemmer if True.
    """
    updates = []
    for j in tqdm(range(len(text_list))):
        
        text = text_list[j]
        
        #LOWERCASE TEXT
        text = text.lower()
        
        #REMOVE NUMERICAL DATA AND PUNCTUATION
        text = re.sub("[^a-zA-Z]", ' ', text)
        
        #REMOVE TAGS
        text = BeautifulSoup(text).get_text()
        
        if lemmatize:
            text = " ".join(lemma.lemmatize(word) for word in text.split())
        
        if stemmer:
            text = " ".join(snowball_stemmer.stem(word) for word in text.split())
        
        updates.append(text)
  
    return updates

def update_df(dataframe, list_updated, coluna):
    dataframe.update(pd.DataFrame({coluna: list_updated}))


def stopwording (text_list, idioma):
  stop_words = set(stopwords.words(idioma))
  listasstopword = []

  for i in range(0, len(text_list)):
    word_tokens = word_tokenize(text_list[i]) 

    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
  
    for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w) 
    filtered_sentence = ((" ").join(filtered_sentence))
    listasstopword.append(filtered_sentence)

  return listasstopword

def preprocessing (dataframeprep, colunas ,idioma, lem, stem): 
  """
  agrega a remoção de stopwords e de lem, stem, noise removal, lowercasing
  """ 
  for i in colunas:
    stopspls = stopwording(dataframeprep[i],idioma)
    update_df(dataframeprep, stopspls, i)

    cleanpls = clean(dataframeprep[i], lem, stem)
    update_df(dataframeprep, cleanpls, i)

#Pre Processing

In [ ]:
# Russian into English 
ruen_prep = ruen
preprocessing(ruen_prep, ["translation", "reference"], "english", True, True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/17980 [00:00<?, ?it/s]

  0%|          | 0/17980 [00:00<?, ?it/s]

#Bleu

## Bleu 4 Avg

In [ ]:
bleu_avg_csen_scores = []
for i in range(csen.shape[0]):
    
    bleu_avg_csen_scores.append(BLEU_4_avg([csen['reference'][i]], csen['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(bleu_avg_csen_scores), np.asarray(csen['z-score'])))
print(np.corrcoef(np.asarray(bleu_avg_csen_scores), np.asarray(csen['z-score']))[0][1])

RuntimeError: ignored

In [ ]:
bleu_avg_deen_scores = []
for i in range(deen.shape[0]):
    
    bleu_avg_deen_scores.append(BLEU_4_avg([deen['reference'][i]], deen['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(bleu_avg_deen_scores), np.asarray(deen['z-score'])))
print(np.corrcoef(np.asarray(bleu_avg_deen_scores), np.asarray(deen['z-score']))[0][1])

RuntimeError: ignored

In [ ]:
bleu_avg_enfi_scores = []
for i in range(enfi.shape[0]):
    
    bleu_avg_enfi_scores.append(BLEU_4_avg([enfi['reference'][i]], enfi['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(bleu_avg_enfi_scores), np.asarray(enfi['z-score'])))
print(np.corrcoef(np.asarray(bleu_avg_enfi_scores), np.asarray(enfi['z-score']))[0][1])

KendalltauResult(correlation=0.32094832427869757, pvalue=0.0)
0.43740027512398855


In [ ]:
bleu_avg_enzh_scores = []
for i in range(enzh.shape[0]):
    
    bleu_avg_enzh_scores.append(BLEU_4_avg([enzh['reference'][i]], enzh['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(bleu_avg_enzh_scores), np.asarray(enzh['z-score'])))
print(np.corrcoef(np.asarray(bleu_avg_enzh_scores), np.asarray(enzh['z-score']))[0][1])

KendalltauResult(correlation=0.004610905951697545, pvalue=0.5673300415882672)
0.03159698190768638


In [ ]:
bleu_avg_ruen_scores = []
for i in range(ruen.shape[0]):
    
    bleu_avg_ruen_scores.append(BLEU_4_avg([ruen['reference'][i]], ruen['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(bleu_avg_ruen_scores), np.asarray(ruen['z-score'])))
print(np.corrcoef(np.asarray(bleu_avg_ruen_scores), np.asarray(ruen['z-score']))[0][1])

KendalltauResult(correlation=0.21327386647483543, pvalue=0.0)
0.30533072546111395


In [ ]:
bleu_avg_zhen_scores = []
for i in range(zhen.shape[0]):
    
    bleu_avg_zhen_scores.append(BLEU_4_avg([zhen['reference'][i]], zhen['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(bleu_avg_zhen_scores), np.asarray(zhen['z-score'])))
print(np.corrcoef(np.asarray(bleu_avg_zhen_scores), np.asarray(zhen['z-score']))[0][1])

KendalltauResult(correlation=0.20195676419040015, pvalue=0.0)
0.2940051440664545


#Rouge

## RougeN

In [ ]:
rouge_fscore_4_avg_csen_scores = []
for i in range(csen.shape[0]):
    rouge_fscore_4_avg_csen_scores.append(rouge_fscore_4_avg([csen['reference'][i]], csen['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(rouge_fscore_4_avg_csen_scores), np.asarray(csen['z-score'])))
print(np.corrcoef(np.asarray(rouge_fscore_4_avg_csen_scores), np.asarray(csen['z-score']))[0][1])

KendalltauResult(correlation=0.261117208279459, pvalue=0.0)
0.37350892366679034


In [ ]:
rouge_fscore_4_avg_deen_scores = []
for i in range(deen.shape[0]):
    rouge_fscore_4_avg_deen_scores.append(rouge_fscore_4_avg([deen['reference'][i]], deen['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(rouge_fscore_4_avg_deen_scores), np.asarray(deen['z-score'])))
print(np.corrcoef(np.asarray(rouge_fscore_4_avg_deen_scores), np.asarray(deen['z-score']))[0][1])

KendalltauResult(correlation=0.19655819227623195, pvalue=0.0)
0.28028564439505627


In [ ]:
rouge_fscore_4_avg_enfi_scores = []
for i in range(enfi.shape[0]):
    rouge_fscore_4_avg_enfi_scores.append(rouge_fscore_4_avg([enfi['reference'][i]], enfi['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(rouge_fscore_4_avg_enfi_scores), np.asarray(enfi['z-score'])))
print(np.corrcoef(np.asarray(rouge_fscore_4_avg_enfi_scores), np.asarray(enfi['z-score']))[0][1])

KendalltauResult(correlation=0.31758844692931865, pvalue=0.0)
0.43428082874633345


In [ ]:
rouge_fscore_4_avg_enzh_scores = []
for i in range(enzh.shape[0]):
    rouge_fscore_4_avg_enzh_scores.append(rouge_fscore_4_avg([enzh['reference'][i]], enzh['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(rouge_fscore_4_avg_enzh_scores), np.asarray(enzh['z-score'])))
print(np.corrcoef(np.asarray(rouge_fscore_4_avg_enzh_scores), np.asarray(enzh['z-score']))[0][1])

KendalltauResult(correlation=0.004573420101617419, pvalue=0.5704167141845551)
0.030209454233655335


In [ ]:
rouge_fscore_4_avg_ruen_scores = []
for i in range(ruen.shape[0]):
    rouge_fscore_4_avg_ruen_scores.append(rouge_fscore_4_avg([ruen['reference'][i]], ruen['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(rouge_fscore_4_avg_ruen_scores), np.asarray(ruen['z-score'])))
print(np.corrcoef(np.asarray(rouge_fscore_4_avg_ruen_scores), np.asarray(ruen['z-score']))[0][1])

KendalltauResult(correlation=0.21040762687637338, pvalue=0.0)
0.3004956554280464


In [ ]:
rouge_fscore_4_avg_zhen_scores = []
for i in range(zhen.shape[0]):
    rouge_fscore_4_avg_zhen_scores.append(rouge_fscore_4_avg([zhen['reference'][i]], zhen['translation'][i]))

#Evaluation
print(stats.kendalltau(np.asarray(rouge_fscore_4_avg_zhen_scores), np.asarray(zhen['z-score'])))
print(np.corrcoef(np.asarray(rouge_fscore_4_avg_zhen_scores), np.asarray(zhen['z-score']))[0][1])

KendalltauResult(correlation=0.19887767343032067, pvalue=0.0)
0.29168226261205


## RougeL

In [ ]:
scorer = rouge_scorer.RougeScorer([ 'rougeL'], use_stemmer=False)

In [ ]:
rouge_L_csen_scores = []
for i in range(csen.shape[0]):
    scores = scorer.score(checoen['reference'][i],
                          checoen['translation'][i])
    rouge_L_csen_scores.append(scores['rougeL'].fmeasure)

#Evaluation
print(stats.kendalltau(np.asarray(rouge_L_csen_scores), np.asarray(csen['z-score'])))
print(np.corrcoef(np.asarray(rouge_L_csen_scores), np.asarray(csen['z-score']))[0][1])

KendalltauResult(correlation=0.3081929203540278, pvalue=0.0)
0.45445445660714384


## RougeS

In [ ]:
def rouge_S(refs, candidate, n, k):
    if len(skip_gram([candidate], n, k)) == 0 or len(skip_gram(refs, n, k)) == 0: #If the translation and reference have less words than n
            return 0
    
    def Rouge_skipgrams_precision(refs, candidate, n, k):

        refs = [skip_gram(refs, n, k)]
        candidate = skip_gram([candidate], n, k)

        return sum([min(count, max([ref[word] for ref in [Counter(ref) for ref in refs]])) for word, count in Counter(candidate).items()])/len(candidate)

    def Rouge_skipgrams_recall(refs, candidate, n, k):

        refs = [skip_gram(refs, n, k)]
        candidate = skip_gram([candidate], n, k)

        return sum([min(count, max([ref[word] for ref in [Counter(ref) for ref in refs]])) for word, count in Counter(candidate).items()])/len(refs)
    
    precision = Rouge_skipgrams_precision(refs, candidate, n, k)
    recall = Rouge_skipgrams_recall(refs, candidate, n, k)
    if precision+recall == 0: #Can´t divide by zero
        return 0
    f1= 2*(precision*recall)/(precision+recall)
    return f1

In [ ]:
rouge_s_csen_scores = []
for i in range(csen.shape[0]):
    rouge_s_csen_scores.append(rouge_S([csen['reference'][i]], csen['translation'][i], 2, 1))

#Evaluation
print(stats.kendalltau(np.asarray(rouge_s_csen_scores), np.asarray(csen['z-score'])))
print(np.corrcoef(np.asarray(rouge_s_csen_scores), np.asarray(csen['z-score']))[0][1])

KendalltauResult(correlation=0.26506138636822224, pvalue=0.0)
0.3823141404564324


#WordMover

## Word2Vec

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api

C:\Users\Miguel\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
model = api.load('word2vec-google-news-300')

In [ ]:
#index 589 da inf, n sei pq
#Provavelmente deviamos tirar pontos finais etc
word2vec_csen_scores = []
for i in range(csen.shape[0]):
    distance = model.wmdistance(csen['reference'][i].lower().split(), csen['translation'][i].lower().split())
    word2vec_csen_scores.append(distance)

#Evaluation
print(stats.kendalltau(np.asarray(word2vec_csen_scores), np.asarray(csen['z-score'])))
print(np.corrcoef(np.asarray(word2vec_csen_scores), np.asarray(csen['z-score']))[0][1])

0
0.35350108500641914
1
0.28181631828144194
2
0.41225617898317396
3
0.4911523182647682
4
0.7893618264641522
5
0.5561314589116002
6
0.636742490686212
7
0.3228896677378502
8
0.14131907711413502
9
0.573835689161703
10
0.8905425546609397
11
0.2949007950959268
12
0.8249754254057167
13
0.3240830745530443
14
0.7050802080262045
15
0.7169837131674268
16
0.5583995544939993
17
0.6258772885723057
18
0.37936190001366304
19
0.6416836893958966
20
0.7721063311484074
21
0.5129411880913298
22
1.2687165889842509
23
0.38372727426973824
24
0.8874364283883165
25
0.5033978476394814
26
0.6879847105039784
27
0.8439710703293103
28
0.0
29
0.4600438609911312
30
1.07053648749547
31
0.5598997624393065
32
0.4390610048352231
33
0.43428389446841653
34
0.6632450644267315
35
0.38939127348777447
36
0.5756956152719537
37
0.2708381202697754
38
0.460179169471002
39
0.6292306906581613
40
0.6060436092231679
41
0.14643119406694802
42
0.563025437709016
43
0.4541839978952224
44
0.39339377547221016
45
0.49651348632185544
46
0.588

0.2832529089770975
387
1.0023518941944085
388
0.5585645253336854
389
0.752661860023346
390
0.4814522295565948
391
0.4159356554851153
392
0.4065643538758234
393
0.5555282555150614
394
0.9875911072963421
395
0.732250772730743
396
0.4210928025367492
397
0.28117434609846237
398
0.6695619616229114
399
0.5554734835418882
400
0.6108354287623337
401
0.5199137184237531
402
0.5787690823753833
403
0.3444793993352468
404
0.24015204492110015
405
0.5945011741138101
406
0.5415491085527989
407
0.8703126359527582
408
0.5215270075144376
409
0.4511142066379159
410
0.4031702272132245
411
0.4577449999152834
412
0.5061367624101949
413
0.2778451974875629
414
0.4234538664287328
415
0.5537590569055659
416
0.6839277514026805
417
0.5217722559149455
418
0.711731027945709
419
0.23525660200694898
420
0.6539266594165474
421
0.7260777364564147
422
0.799404617710264
423
0.6453912541499683
424
0.5427442794428872
425
0.33156816049883425
426
0.6504482036451101
427
0.8567048991741899
428
0.5789327991208431
429
0.460257533

0.4720210833552287
783
0.5675681777627918
784
0.6204965575352137
785
0.6072621791953183
786
0.3989916846727283
787
0.5361171263289461
788
0.5562408120637966
789
0.7449524241622552
790
0.6265238212186255
791
0.5079562980175344
792
0.554851675649341
793
0.4966233347897766
794
1.1626348566926954
795
0.42047436867825516
796
0.5227316337338368
797
0.4820808177502612
798
0.5202105145562488
799
0.450847136517351
800
0.6333048818671405
801
0.4311297750040516
802
0.4670167893195428
803
0.31298764401349055
804
0.4289328232175218
805
0.7176421265531127
806
0.878718883262315
807
0.47508353746607435
808
1.1857895735900923
809
0.603056670963625
810
0.3580463177500753
811
0.3973118492562291
812
0.3808339288093035
813
0.734352686833151
814
0.6757610875486477
815
0.0
816
0.2669920161326406
817
0.7851192656172284
818
1.032003857731821
819
0.7060121951907682
820
0.556559078947725
821
0.4660026037708091
822
0.5624607501952179
823
0.5116936392730658
824
0.6775513951082324
825
0.4142292090114802
826
0.44897

0.9594436544836271
1153
0.7379337528739965
1154
0.5652445656195922
1155
0.6171605922051119
1156
0.9076075070495871
1157
0.4846359756667936
1158
0.5988605392135622
1159
0.22549753300945097
1160
0.6820849799282509
1161
0.5334811572546904
1162
0.43066685276300903
1163
0.41999227092205127
1164
0.43558744152822243
1165
1.1630727365083695
1166
0.7791707432646474
1167
0.1855916027416966
1168
0.731388898856986
1169
0.5974000798680318
1170
0.17707298576806216
1171
0.2322082447436536
1172
0.631038595156584
1173
0.17031402142410249
1174
0.6458269569265148
1175
0.5192012188253363
1176
0.26673128037229626
1177
0.7139947791640889
1178
0.5973821004345864
1179
0.4552946204912456
1180
0.3776093109313641
1181
0.6921507665663361
1182
0.5831282527839049
1183
0.2512933787660685
1184
0.4972833901603994
1185
0.17753634071077762
1186
0.8162065982162594
1187
0.32621198605292134
1188
0.3774082780283799
1189
0.7676436933092652
1190
0.8094629873132465
1191
0.8030715164683164
1192
0.4249260101403517
1193
0.5369313

0.523011458499139
1522
0.0
1523
0.7078524399898772
1524
0.571592714732589
1525
0.2179258430232525
1526
0.6418076730769574
1527
0.547291833266163
1528
1.2487699909180403
1529
0.5654033290845369
1530
0.3519629924483605
1531
0.5808145737201053
1532
0.4554924660484892
1533
0.5235771518964376
1534
0.4219640794416654
1535
0.7594622769406637
1536
0.6054470648149078
1537
0.9735752298658843
1538
0.6472616822829036
1539
0.5379384339262326
1540
0.7464382797558308
1541
0.36844831429561975
1542
0.3861081837408922
1543
0.5459247350321154
1544
0.6681614801427576
1545
0.8140490502780907
1546
0.8578681509224869
1547
0.6624991262694276
1548
0.2891375209407806
1549
0.8437940230800777
1550
0.6645158820375527
1551
0.7728995669098941
1552
0.573124683540808
1553
1.1734833124370576
1554
0.8132800809872658
1555
0.9457210617715547
1556
0.619820059548355
1557
0.6753222350023377
1558
0.29624839168787004
1559
0.8246050125358974
1560
0.37734656538752387
1561
0.31887131392671997
1562
0.29879952663046194
1563
0.38296

...

0.7112581280582505
10118
0.7825345562816437
10119
0.4033876059488889
10120
0.6596458752059585
10121
0.37137920980021955
10122
0.2998101833561004
10123
0.5948098467879749
10124
0.5517803993473241
10125
0.48417946711380383
10126
0.5977208351166525
10127
0.8050583031054438
10128
0.36679518893603424
10129
0.6712557598131018
10130
0.7036848203388577
10131
0.5235812889620952
10132
0.6314291135200294
10133
0.5718997699036302
10134
0.7083872042898027
10135
0.5455422338182461
10136
0.5709234571052363
10137
0.0
10138
0.47973485584527376
10139
0.8501870038065201
10140
0.8568760254707541
10141
0.6545777716089723
10142
0.8520109492984235
10143
0.7101103880654221
10144
0.683828406785376
10145
0.4528809481046131
10146
0.4925416180512788
10147
1.2487699909180403
10148
0.379229257255507
10149
0.5928821397304688
10150
0.4084691332830675
10151
0.7103468388027104
10152
0.7594622769406637
10153
0.6570337042619975
10154
0.42255890448854366
10155
0.9735752298658843
10156
0.32417757323601065
10157
0.622771085

0.44080620683110316
10466
0.5464831522340727
10467
0.3429421360992441
10468
0.4718635009127896
10469
0.46133470219354106
10470
0.5684870743684785
10471
0.5586551125931944
10472
0.8169813008439437
10473
0.6115645851601972
10474
0.806392502833161
10475
0.7815665921395871
10476
0.5723442958939018
10477
0.8857820852428913
10478
0.5929856881527604
10479
0.31607500698100216
10480
0.8620959422014819
10481
0.4095168750093874
10482
0.5740189001403924
10483
0.42778350080420363
10484
0.4456399016956162
10485
0.0
10486
0.34197746773122434
10487
0.14010832925587421
10488
0.5085950358094096
10489
0.4978409677839909
10490
0.4012522478235574
10491
0.34082553721441294
10492
1.1162283385816023
10493
0.6894725832228763
10494
0.5680160326077379
10495
0.5666053935900665
10496
0.0
10497
0.4980554879400395
10498
0.692657184216837
10499
0.5885702503508506
10500
0.8003958818419054
10501
0.44950906942814184
10502
inf
10503
0.5609130270269513
10504
0.1392001406076539
10505
0.3867110894215704
10506
0.395069065472

0.6935966617959604
10815
0.7769997139536182
10816
0.3408483687464562
10817
0.4244949075964916
10818
0.0
10819
0.5361069536157657
10820
0.44606310064532617
10821
0.44251732760096285
10822
0.16753335093549024
10823
0.3597559203174005
10824
0.5270567145829848
10825
0.4313524315960206
10826
0.5517485034719569
10827
0.4443663814292671
10828
0.7087678911113253
10829
0.13205879689009228
10830
0.45582978144718694
10831
0.4583741491255542
10832
0.438608377737125
10833
0.9345437174946786
10834
0.5745465192520371
10835
0.3485635267488232
10836
0.7617745395879373
10837
0.640316926282045
10838
0.7500188880900146
10839
0.4259792296906779
10840
0.5035254026572796
10841
0.7638596506944056
10842
0.6714856537486574
10843
0.8069005059944465
10844
0.3567089564536041
10845
0.0
10846
0.2711228303257668
10847
0.392084934260284
10848
0.5142132700622252
10849
0.6290516771027351
10850
0.8081448857972715
10851
0.5353232743863754
10852
0.26511904667575337
10853
0.6198938311120255
10854
0.9650494869709412
10855
0.

0.40749352996243327
11178
0.6170983670469607
11179
0.8262197323974482
11180
0.3815304753348949
11181
0.7209522456471725
11182
0.6085544975681358
11183
0.36107293540536467
11184
0.7364525903781505
11185
0.5267775779130615
11186
0.9683811137592692
11187
0.7141969404604627
11188
0.4267235807920535
11189
0.4248921073932686
11190
0.5476847532233338
11191
0.5974417487861514
11192
0.23350895569635038
11193
0.3947758957945171
11194
0.750899166039257
11195
0.6250185029870338
11196
0.18825814914285915
11197
0.0
11198
0.6948903667637915
11199
0.4342493147594756
11200
0.7843562133186084
11201
0.38188213422050876
11202
1.0414947582075986
11203
0.6963840924391722
11204
0.622794518828502
11205
0.884249547339764
11206
0.6748954010392152
11207
0.540620892472283
11208
0.7549810701994034
11209
0.6449515534230288
11210
0.4616012378794464
11211
0.5014962901219542
11212
0.460080070554018
11213
0.7551168618617649
11214
0.5063140929587606
11215
0.43321377186533405
11216
0.425311301036096
11217
0.3742961547087

0.49765479163906784
11532
0.7881502007743287
11533
0.20546461675006303
11534
0.6844087226483242
11535
0.7260757434985474
11536
1.353359462100315
11537
0.5057785811633176
11538
0.7505641932470977
11539
0.7642731175366393
11540
0.6340016542872685
11541
0.3633250133204243
11542
0.4092504498271646
11543
0.8136412680609376
11544
0.5290935355284333
11545
0.5034749570335029
11546
0.6402807086562592
11547
0.5381241160323297
11548
0.6686099057191335
11549
0.7539302787019432
11550
0.1769078289760183
11551
0.4712212096214145
11552
0.5743346879311354
11553
0.559760208552534
11554
0.9118056967324959
11555
0.796427620348394
11556
0.9043189614890668
11557
1.1402612480573426
11558
0.5695878448387605
11559
0.5425430145535104
11560
1.0618347822848482
11561
0.5370598988599002
11562
0.5375529681129153
11563
0.4712072534226666
11564
0.49831141753781816
11565
0.561408683588791
11566
0.3632990569970896
11567
0.6811788429936163
11568
0.2918341366559106
11569
0.8149080708660351
11570
0.8120604753846357
11571
0

# Bert

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity #Tem mesmo de ser cosine similarity? N pode ser outra?

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

Some weights of the model checkpoint at C:\Users\Miguel/.cache\torch\sentence_transformers\sbert.net_models_bert-base-nli-mean-tokens\0_BERT were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Demora mesmo bue a correr, provavelmente deixar este para ultimo a implementar
#Ver qual implementaçao de bert é esta, e ver quais as diferenças das outras e como as implementar

bert_csen_scores = []
for i in range(csen.shape[0]):
    sentences = [csen['reference'][i].lower(), csen['translation'][i].lower()]
    sentence_embeddings = model.encode(sentences)
    value = cosine_similarity(
            [sentence_embeddings[0]],
            sentence_embeddings[1:]
            )[0][0]
    print(value)
    bert_csen_scores.append(value)

#Evaluation
print(stats.kendalltau(np.asarray(bert_csen_scores), np.asarray(csen['z-score'])))
print(np.corrcoef(np.asarray(bert_csen_scores), np.asarray(csen['z-score']))[0][1])

0.80537224
0.9408244
0.9336711
0.857001
0.84484625
0.9206691
0.92013466
0.94927514
0.99427736
0.9179299
0.87093943
0.9705784
0.5094472
0.94132775
0.8642999
0.8459557
0.900591
0.85898864
0.9706361
0.867621
0.9224732
0.9429828
0.9979851
0.9677232
0.89970756
0.94374645
0.88889253
0.85734904
0.9564291
0.9520824
0.7989466
0.9636147
0.9158299
0.9133125
0.8637452
0.9487923
0.97569215
0.97122943
0.79882765
0.91176087
0.9535755
0.98711634
0.9712315
0.9251042
0.96511817
0.90115345
0.9489025
0.92712104
1.0
0.7547711
0.92206573
0.99168926
0.8756236
0.85060203
0.9825605
0.90108246
0.9772761
0.88781595
0.86551297
0.9902766
0.9531883
0.80820113
0.990995
0.9456944
0.97380346
0.85920215
0.7721099
0.9633376
0.86331165
0.7684773
0.92113507
0.9582691
0.74898094
0.9292048
0.96537864
0.7411212
0.8552923
0.8764492
0.9071355
0.98076326
0.91107106
0.7955589
0.9589298
0.97285366
0.92558885
0.82856685
0.8862405
0.93229634
0.98304474
0.84840566
0.5658022
0.8048928
0.9050734
0.69561666
0.80512774
0.6365263
0.43812

0.8790524
0.8695301
0.8308579
0.9498416
0.9181421
0.92226934
0.8782568
0.7960147
0.9043869
0.91964155
0.83010185
0.97922385
0.8971763
0.8382703
0.593559
0.9443073
0.8140687
0.9856499
0.9201369
0.5799532
0.97287226
0.81538856
0.88009
0.93075776
0.98294723
0.97629976
0.9122895
0.9131217
0.9203649
0.90227383
0.9444471
0.87200195
0.9090565
0.8301545
0.8111658
0.8443039
0.8777939
0.78332275
0.9321805
0.92118126
0.97084844
0.6178733
0.8519521
0.93905306
0.94375813
0.47543436
0.99285674
0.77434576
0.7581347
0.96287435
0.9676246
0.8243288
0.8434357
0.83798116
0.91625077
0.66077375
0.7222823
0.93305075
0.9530904
0.93221647
0.9515178
0.8132342
0.80018574
0.96408033
0.94509375
0.5482588
0.8917267
0.8757622
0.971519
0.9025916
0.9325415
0.951385
0.84195113
0.83233
0.9866253
0.97060335
0.98203254
0.8440583
0.9894908
0.96092176
0.8743529
0.9751346
0.8673734
0.8290654
0.9609095
0.3195147
0.98822474
0.9394411
0.81809306
0.87115973
0.7750344
0.89884365
0.14784689
0.8929062
0.86723363
0.967538
0.8727764


0.9141647
0.94871557
0.84732044
0.8375127
0.9639282
0.9824962
0.9660051
0.89992595
0.4705286
0.97544307
0.7303053
0.8797678
0.7771944
0.96379477
0.937238
0.98135626
0.92885804
0.91423696
0.92858106
0.8578206
0.9387293
0.94897026
0.91209334
0.92070925
0.7655053
0.9607355
0.9645944
0.8839422
0.95001435
0.9251509
0.8284854
0.96574914
0.8939897
0.8980043
0.96045846
0.98201096
0.90844417
0.8060615
0.91997766
0.9653119
0.9871278
0.83966494
0.95852625
0.94119436
0.9298283
0.9188293
0.94802344
0.88024235
0.92795765
0.9795264
0.8382474
0.9584734
0.68505925
0.823758
0.81731987
0.9696401
0.87614566
0.84719956
0.9275458
0.91680014
0.83332056
1.0
0.86032903
0.93945503
0.896096
0.7181067
1.0
0.84580433
0.6351538
0.9683001
0.92791224
0.9121673
0.9705851
0.9622878
0.9763532
0.82241035
0.7340759
0.3185198
0.9645304
0.94165885
0.9369949
0.91363335
0.83265674
0.8791162
0.9331417
0.52726233
0.8969922
0.95171666
0.8862608
0.8758656
0.83567965
0.91732144
0.84133863
0.9891468
0.9867755
0.86172915
0.91885173


0.8499162
0.9736397
0.94144654
0.89760005
0.97056854
0.8700645
0.9714339
0.6924122
0.9236556
0.9251481
0.92932916
0.8971865
0.91346586
0.91440487
0.87887156
0.9542899
0.80643284
0.7346323
0.84817064
0.8149313
0.670567
0.9462695
0.93208396
0.90560955
0.8863094
0.87640667
0.97947633
0.9166279
0.8381396
0.92103
0.9231439
0.9497998
0.6829996
0.73964953
0.87994474
0.78068185
0.92510045
0.85169876
0.98244643
0.90502703
0.8493714
0.8326919
0.9634794
0.91181654
0.82878196
0.97957575
0.92257845
0.93799907
0.6183702
0.8078321
0.890296
0.8967184
0.82995653
0.988265
0.9419434
0.9037243
0.91952753
0.54797477
0.7476746
0.9203584
0.76534647
0.9607415
0.81178534
0.9569131
0.8766804
0.89691955
0.9197785
0.94333065
0.8607904
0.8828
0.95644855
0.9058795
0.9343411
0.8350966
0.91052556
0.8244277
0.86229634
0.6410613
0.9507002
0.79275036
0.93508244
0.86837655
0.96715736
0.8415493
0.7218189
0.96437085
0.8903377
0.9004033
0.9520217
0.51746905
0.79832006
0.84967405
0.9903324
0.94285
0.9475334
0.97027194
0.7454

0.62870395
0.8494136
0.904159
0.87701654
0.76279175
0.77400947
0.96017045
0.93256223
0.86427456
0.8454572
0.69128114
0.8794368
0.9539478
0.7788452
0.85132074
0.7680669
0.76571774
0.9465487
0.44687814
0.8806821
0.8951119
0.95690686
0.94646865
0.9024498
0.77308714
0.855253
0.97732824
0.789348
0.8762581
0.7223175
0.98720425
0.9597168
0.9180162
0.7746831
0.94449437
0.9747978
0.9068303
0.2810399
0.76655334
0.69860446
0.9338486
0.5133869
0.9419353
0.8828011
0.96277094
0.7556569
0.7748486
0.9081018
0.91594326
0.9226306
0.7525555
0.95190644
0.63640356
0.9376044
0.93985605
0.88295805
0.965835
0.5920408
0.8227383
0.96358895
0.89081234
0.7995584
0.94155735
0.797992
0.62436664
0.67670214
0.98153687
0.9447948
0.86606705
0.96892345
0.9610263
0.9710184
0.7967104
0.9597577
0.88734734
0.5670438
0.9635713
0.92508966
0.9624521
0.8867247
0.8404483
0.7836169
0.9017899
0.97870994
0.88839155
0.96239734
0.79280484
0.75770485
0.9469272
0.9047954
0.8369307
0.9191976
0.97875094
0.9531509
0.7981742
0.80034935
0.9

...

0.99175066
0.8909689
0.95508325
0.9812652
0.85504603
0.95093995
0.94642615
0.88962984
0.67119086
0.93962485
0.94322443
0.9670499
0.97793055
0.8932613
0.7854653
0.95745885
0.91585565
0.95582664
0.77997565
0.9397931
0.94764733
0.95162594
0.9688707
0.92137957
0.86172247
0.9607334
0.91546667
0.9881712
0.66103363
0.9583382
0.9510968
0.96934193
0.97347474
0.9842584
0.9254635
0.8847574
0.6295627
0.98229057
0.9372307
0.9545601
0.9521431
0.9289192
0.7709721
0.9449358
0.74253017
0.97250605
0.9430523
0.94806904
0.9430672
0.7618802
0.99260724
0.92880833
0.87682927
0.8974666
0.87580305
0.84066117
0.96751463
0.52096546
0.94797075
0.96710724
0.9741014
0.8501266
0.97861034
0.9585004
0.81277925
0.984227
0.94877875
0.9003004
0.7843195
0.86278856
0.87404186
0.81125295
0.8178789
0.9701562
0.9516451
0.9744634
0.9479174
0.87671036
0.90149325
0.88068795
0.95597595
0.97736526
0.9678771
0.9295565
0.9464359
0.98436683
0.9583858
0.8945795
0.941977
0.8416829
0.8218765
1.0
0.8262791
0.9804148
0.97901165
0.98877454

0.9330095
0.9779939
0.7608274
0.89017403
0.9440516
0.698419
0.9450174
0.888296
0.90394914
0.9724171
0.80989283
0.9793056
0.97684366
0.8723181
0.9281398
0.88848317
0.94400924
0.9450607
0.76707923
0.735538
0.7538845
0.92974484
0.94260836
0.9328951
0.8667253
0.9688142
0.906986
0.97261703
0.9551397
0.90719515
0.803074
0.97211826
0.988577
0.98314166
0.9732178
0.9342485
0.8651502
0.68932176
0.98205113
0.941064
0.93645674
0.963665
0.97888374
0.9661857
0.9725311
0.82049054
0.9751835
0.85709983
0.9235754
0.9676246
0.8254709
0.9048404
0.6425128
0.8601393
0.94509757
0.97118866
0.71109986
0.9132695
0.91186893
0.92994535
0.9607381
0.890736
0.96107084
0.86435115
0.9068669
0.81662685
0.9189536
0.96348083
0.8911009
0.8127435
0.9267325
0.9573296
0.8846896
0.8926038
0.56557524
0.9471348
0.946671
0.5549576
0.96326554
0.87140316
0.8663289
0.8941184
0.97772133
0.93501955
0.8516519
0.8479169
0.991982
0.77727115
0.8043292
0.9371469
0.97556543
0.9239088
0.97205925
0.96621346
0.6422765
0.8906798
0.91279405
0.8

0.94609934
0.8955106
0.8157604
0.78518146
0.9675774
0.8564128
0.9250811
0.7694429
0.8862565
0.9205552
0.63924783
0.9671789
0.7923437
0.93677413
0.95614916
0.902354
0.91551787
0.9752092
0.9811629
0.72512996
0.9698785
0.99250996
0.8193151
0.8129108
0.88961756
0.9749684
0.97749186
0.9951955
0.99441487
0.99342847
0.92484355
0.5975967
0.91760445
0.74546576
0.93321395
0.95735395
0.7077763
0.7805822
0.9407352
0.82387537
0.8638574
0.8813746
0.93340445
0.94048333
0.90335983
0.9750836
0.92989683
0.8799317
0.96311045
0.93783414
0.9524947
0.93124366
0.99002874
0.8286934
0.92300296
0.95490956
0.97392905
0.9584507
0.7420562
0.9458224
0.9999998
0.9498824
0.69184244
0.87870264
0.9725176
1.0
0.9905051
0.93834186
0.9555937
0.8338148
0.96132755
0.9457563
0.98262554
0.98743194
0.9987252
0.9346057
0.76755816
0.9953666
0.9805078
0.9248411
0.8888028
0.9836048
0.95918703
0.97816443
0.8941105
0.89469033
0.9224531
0.9732516
0.81991136
0.92790127
1.0000002
0.88189435
0.9120315
0.88416195
0.89749956
0.9264795
0.9

0.9652279
0.8575916
0.9751607
0.9327873
0.95944214
0.88476324
0.9186134
0.95064354
0.56465304
0.9276786
0.8856567
0.755715
0.9865217
0.8294749
0.9229367
0.8247868
0.9280996
0.95327616
0.89869344
0.7216668
0.9444036
0.95788395
0.99668026
0.91542315
0.71204954
0.97595197
0.9592346
0.89060414
0.99554026
0.9006324
0.9607985
0.9361079
0.9661174
0.5833432
0.86069554
0.88774395
0.83128965
0.9979283
0.93620884
0.8759178
0.77705526
0.9693776
0.8701006
0.97324157
0.8803513
0.8319697
0.73881483
0.95455664
0.9199649
0.9313632
0.9337314
0.8062316
0.93668675
0.9079037
0.7269441
0.8616922
0.72759914
0.9844043
0.91171885
0.9283093
0.9277578
0.9577332
0.8949255
0.94937205
0.93870664
0.9423302
0.63131696
0.9695945
0.5150304
0.9315367
0.95374525
0.9510347
0.93509984
0.9430784
0.96182275
0.8406185
0.80746776
0.9089026
0.99590445
0.83123755
0.8930022
0.9199765
0.9154927
0.96869576
0.7520403
0.8883451
0.81576335
0.97542477
0.9355017
0.77052605
1.0
0.7953315
0.8840065
0.98843694
0.73691213
0.8886315
0.943374

0.80097806
0.8810682
0.95826584
0.81926066
0.849803
0.9226308
0.8957685
0.96758485
0.937676
0.9404398
0.9712238
0.9334421
0.85726804
0.9520065
0.76940924
0.96834135
0.88361776
0.8587622
0.9029864
0.9211165
0.9922528
0.8850092
0.9598596
1.0000002
0.9839978
0.79399276
0.95974016
0.96933883
0.8525231
0.9507067
0.83371997
0.7452389
0.57028335
0.9922673
0.6426114
0.9207146
0.98633015
0.91315943
0.8795957
0.79903495
0.9667741
0.86901283
0.8464473
0.9153856
0.9281938
0.80301785
0.8609465
0.94830894
0.91511476
0.94961894
0.9318164
0.8936068
0.9810591
0.98481846
0.84213173
0.9419155
0.87554234
0.93103075
0.9574484
0.928848
0.72741514
0.9067543
0.697667
0.9378661
0.9279344
0.95191
0.8278775
0.98492414
0.9866859
0.892853
0.8277583
0.99212104
1.0
0.8696502
0.94092137
0.90363204
0.97508824
0.6540497
0.9279292
0.9565355
0.8984024
0.8181356
0.8453711
0.76550114
0.8958214
0.95535827
0.9230692
0.796579
0.8011894
0.9691668
0.9128344
0.97182596
0.93266934
0.78914833
0.9222133
0.9118851
0.9334914
0.774315

# Comet

In [ ]:
pip install torch==1.4.0 torchvision==0.5.0 -f https://download.pytorch.org/whl/cu100/torch_stable.html

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from comet.models import download_model

ImportError: ignored

In [ ]:
model = download_model("wmt-large-da-estimator-1719")

In [ ]:
data = [
    {
        "src": "Dem Feuer konnte Einhalt geboten werden",
        "mt": "The fire could be stopped",
        "ref": "They were able to control the fire."
    },
    {
        "src": "Schulen und Kindergärten wurden eröffnet.",
        "mt": "Schools and kindergartens were open",
        "ref": "Schools and kindergartens opened"
    }
]
model.predict(data, cuda=True, show_progress=True)

# Laser

Ta mal implementado de certeza, era so pa ver se conseguia aceder aos documentos

In [ ]:
ls_csen_ref = np.load('C:/Users/Eurico/Documents/TextMining/Project/LaserEmbeddings/corpus/cs-en/laser.reference_embeds.npy')
ls_csen_tran = np.load('C:/Users/Eurico/Documents/TextMining/Project/LaserEmbeddings/corpus/cs-en/laser.translation_embeds.npy')

In [ ]:
cosine_similarity(
            ls_csen_ref[0].reshape(-1,1),
            ls_csen_tran[0].reshape(-1,1)
            )[0][0]

1.0

#CHRF

In [26]:
def crhf(dataset, beta,max_len):
  chrf_list=[]
  for i in range(len(dataset)):

    chrf_list.append(nltk.translate.chrf_score.sentence_chrf(dataset['reference'][i], dataset['translation'][i],max_len=max_len, beta=beta))
  return chrf_list

#ALBERT

In [29]:
!pip install tensorflow==1.15.2
!pip install tensorflow_hub==0.7
!pip install tensorflow-gpu==1.15
!pip install sentencepiece

     |████████████████████████████████| 110.5MB 91kB/s 
     |████████████████████████████████| 3.8MB 44.5MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 512kB 56.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=9c63ebd4719ff75d78c3e10fc16bb5a267047c5df3d6327b37e5e633e5b22d37
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: unbabel-comet 0.1.0 has requirement tensorboard==2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: pytorch-lightning 1.3.0 has requirement fsspec[http]>=2021.4.0, but you'll have fsspec 0.8.7 which is incompatible.
ERROR: pytorch-lightning 1.3.0 has requirement tensorboard!=2.5.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
  Found 

In [38]:
!pip install albert
!pip install run_pretaining

ERROR: Could not find a version that satisfies the requirement run_pretaining (from versions: none)
ERROR: No matching distribution found for run_pretaining


In [37]:
import albert
from albert import run_pretaining

ImportError: ignored

In [43]:
model=albert

#RoBERTa

In [56]:
!pip install regex requests
!pip install hydra-core
!pip install omegaconf

In [57]:
 !apt-get install python3.9-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.9 libpython3.9-dev libpython3.9-minimal libpython3.9-stdlib
  python3.9 python3.9-minimal
Suggested packages:
  python3.9-venv python3.9-doc binfmt-support
The following NEW packages will be installed:
  libpython3.9 libpython3.9-dev libpython3.9-minimal libpython3.9-stdlib
  python3.9 python3.9-dev python3.9-minimal
0 upgraded, 7 newly installed, 0 to remove and 34 not upgraded.
Need to get 11.7 MB of archives.
After this operation, 46.8 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.9-minimal amd64 3.9.5-1+bionic1 [787 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.9-minimal amd64 

In [64]:
!pip install fairseq

In [65]:
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


DistutilsClassError: ignored